In [6]:
from langchain_community.document_loaders import PyPDFLoader
# import chroma

import os
from dotenv import load_dotenv

load_dotenv()

key = os.getenv("gen_api_key")



path = "../assignments/Ayyub_Raji.pdf"
loader = PyPDFLoader(path)

pages = loader.load()

In [7]:
if pages:
    for page in pages:
        print(page.page_content)

Ayyub  Raji  
Frontend  Engineer  |  Backend  Developer  |  AI  Engineering  Trainee  
Lagos,  Nigeria  •  ayyubraji87@gmail.com  •  GitHub:  
github.com/raji-ayyub
 
•
 
LinkedIn:
 
linkedin.com/in/ayyub-raji
 
PROFESSIONAL  SUMMARY  
Full-stack  engineer  with  strong  experience  in  Next.js ,  TypeScript ,  TailwindCSS ,  and  FastAPI .  
Builds
 
clean,
 
scalable
 
interfaces
 
and
 
solid
 
backend
 
architectures.
 
Currently
 
expanding
 
into
 
AI
 
engineering
,
 
focusing
 
on
 
LLM
 
integration,
 
vector
 
search,
 
prompt
 
engineering,
 
and
 
custom
 
AI
 
system
 
development.
 
Known
 
for
 
fast
 
learning,
 
ownership,
 
and
 
delivering
 
solutions
 
end-to-end.
 
 
CORE  SKILLS  
Frontend  Engineering  
●  Next.js  (App  Router),  React,  TypeScript,  TailwindCSS,  ShadCN/UI,  Axios  
Backend  Engineering  
●  FastAPI,  Python,  MongoDB,  Pydantic  models,  CMS-style  backend  design,  
Authentication,
 
REST
 
API
 
design.
 
 
AI  Engineering  and  ML  
●  LLM 

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
    chunk_overlap=200)

document = splitter.split_documents(documents=pages)
document

[Document(metadata={'producer': 'Skia/PDF m145 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Untitled document', 'source': '../assignments/Ayyub_Raji.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='Ayyub  Raji  \nFrontend  Engineer  |  Backend  Developer  |  AI  Engineering  Trainee  \nLagos,  Nigeria  •  ayyubraji87@gmail.com  •  GitHub:  \ngithub.com/raji-ayyub\n \n•\n \nLinkedIn:\n \nlinkedin.com/in/ayyub-raji\n \nPROFESSIONAL  SUMMARY  \nFull-stack  engineer  with  strong  experience  in  Next.js ,  TypeScript ,  TailwindCSS ,  and  FastAPI .  \nBuilds\n \nclean,\n \nscalable\n \ninterfaces\n \nand\n \nsolid\n \nbackend\n \narchitectures.\n \nCurrently\n \nexpanding\n \ninto\n \nAI\n \nengineering\n,\n \nfocusing\n \non\n \nLLM\n \nintegration,\n \nvector\n \nsearch,\n \nprompt\n \nengineering,\n \nand\n \ncustom\n \nAI\n \nsystem\n \ndevelopment.\n \nKnown\n \nfor\n \nfast\n \nlearning,\n \nownership,\n \nand\n \ndelivering\n \nsolut

#### EMBEDDINGS


In [9]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=key,
)

test_embedding= embeddings.embed_query("What is RAG?")
print(f"Embedding Dimension: {len(test_embedding)}")
print(f"first five: {test_embedding[:5]}")

Embedding Dimension: 1536
first five: [0.0006491607055068016, 0.025743018835783005, 0.007129499223083258, 0.03339380398392677, -0.03199847787618637]


VECTOR STORE

In [15]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=document,
    embedding=embeddings,
    persist_directory="./chroma_db_baseline"
)
print(f"vector store created with {len(document) }chunks")

query = "Who is Ayyub?"
results = vectorstore.as_retriever(search_kwargs={"k": 5})


print(f"\nQUERY: {query}")
print(results)

ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate



llm = ChatOpenAI(
    model = "gpt-3.5-turbo",
    temperature=0,
    openai_api_key=key
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant to answer questions relating to Ayyub. Answer ONLY using the provided context."),
    ("human", "Question: {question}\n\nContext:\n{context}")
])

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 4}
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
from langchain_core.runnables import RunnablePassthrough


In [ ]:
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)


In [ ]:
response = rag_chain.invoke("Who is Ayyub?")
print(response.content)


Ayyub is a Full-stack engineer with experience in Next.js, TypeScript, TailwindCSS, and FastAPI. He is currently expanding into AI engineering, focusing on LLM integration, vector search, prompt engineering, and custom AI system development. Ayyub has skills in Frontend Engineering, Backend Engineering, and AI Engineering, and he is known for fast learning, ownership, and delivering solutions end-to-end.
